# Configuracion

### Carga de librerias

In [1]:
from Metadatos import generaPathProyecto, RMSLE
from CargaDatos import retornaSets
from Preparacion import preparaDf, separacionEntrenaObjetivo
from Modelo import definirModelo

In [2]:
from CargaDatos import generaPath

### Definicion de Funciones

# Ejecucion de rutina

In [3]:
path_proyecto = generaPathProyecto()
sets_df = retornaSets(path_proyecto = path_proyecto)

In [4]:
prepDf = preparaDf()

In [5]:
df = prepDf.preparar(sets_df)

In [6]:
X_train, X_test, y_train, y_test = separacionEntrenaObjetivo(df, semilla = 1962, prop_prueba= 0.3)

In [7]:
modelo = definirModelo()

In [17]:
y_train.values.reshape(y_train.shape[0],)

array([ 21.90593302,  12.34121398,   6.60182614, ...,   5.06612954,
        18.21414065,  53.61862422])

In [18]:
modelo.fit(X = X_train.values, y = y_train.values.reshape(y_train.shape[0],))

GridSearchCV(cv=None,
       estimator=Pipeline(steps=[('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('escalador', StandardScaler(copy=True, with_mean=True, with_std=True)), ('decisor', RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_fea...ples_split=2, n_estimators=10, n_jobs=1, oob_score=False,
           random_state=1962, verbose=0))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'decisor__n_estimators': [10, 20], 'decisor__min_samples_split': [1, 2, 4, 8, 16], 'decisor__min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(RMSLE, greater_is_better=False), verbose=0)

In [9]:
modelo.best_params_

{'decisor__min_samples_leaf': 2,
 'decisor__min_samples_split': 1,
 'decisor__n_estimators': 20}

In [10]:
modelo.best_score_

-0.35841121521579467

In [11]:
RMSLE(y_train, modelo.predict(X_train))

0.15728725403350383

In [12]:
RMSLE(y_test, modelo.predict(X_test))

0.27506811374588047

In [19]:
X_envio = prepDf.preparar(sets_df, train_o_envio= 'test')

In [20]:
y_envio = modelo.predict(X_envio.values)

In [21]:
import pandas as pd

In [22]:
y_envio = pd.DataFrame(data= y_envio, index= range(1,y_envio.shape[0] + 1), columns= ['cost'])
y_envio.index.name = 'id'

In [23]:
y_envio.to_csv(path_or_buf= generaPath(path_proyecto, 'y_envio3.csv'))

In [ ]:
resultados = validarModelo(modelo)

In [ ]:
registrarModelo(modelo)

In [ ]:
generarEnvio(modelo, input)